In [1]:
import argparse
import os
import random
import shutil
import time
import warnings
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
# from resnet import *

from main import *
from gated import *
from resnet_gated import *


In [2]:
# args = parser.parse_args(args=[])
# args = parser.parse_args()
import easydict 
args = easydict.EasyDict({ "batch-size": 256, 
                          "epochs": 100, 
                          "data": 0, 
                          'arch':'resnet18',
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0})


In [3]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
# device = 'cpu'
device = 'cuda'

1


In [4]:
# print("=> using pre-trained model '{}'".format('resnet18'))
# model = models.__dict__['resnet18'](pretrained=True)
# model = models.resnet18(pretrained=True)
model = resnet18(pretrained=False)
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)


dilation [False, False, False]
stride 1
downsample
stride 2
downsample
stride 2
downsample
stride 2


In [5]:
# model_dict = model.state_dict() 
# for k in model_dict :
#     print(k)
#     print(model_dict[k].shape)

In [6]:
# model.eval()

In [7]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [9]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None

In [10]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=96, 
    shuffle=(train_sampler is None),
    num_workers=12, 
#     pin_memory=True, 
    sampler=train_sampler)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, 
    shuffle=False,
    num_workers=12, )
#     pin_memory=True)

In [11]:
# model.to(device)

In [ ]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    adjust_learning_rate(optimizer, epoch, args)

    # train for one epoch
    epoch_loss = train(train_loader, model, criterion, optimizer, epoch, args)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, args)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)    

Epoch: [0][    0/13346]	Time  1.547 ( 1.547)	Data  1.191 ( 1.191)	Loss 7.0724e+00 (7.0724e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
 * Acc@1 16.246 Acc@5 36.214
************train_loss 5.311251977296421 val_acc 16.246179580688477*************
Epoch: [1][    0/13346]	Time  1.467 ( 1.467)	Data  1.397 ( 1.397)	Loss 4.3883e+00 (4.3883e+00)	Acc@1  15.62 ( 15.62)	Acc@5  34.38 ( 34.38)
 * Acc@1 25.315 Acc@5 49.208
************train_loss 4.132702326910366 val_acc 25.31511688232422*************
Epoch: [2][    0/13346]	Time  1.447 ( 1.447)	Data  1.364 ( 1.364)	Loss 3.5355e+00 (3.5355e+00)	Acc@1  26.04 ( 26.04)	Acc@5  58.33 ( 58.33)
 * Acc@1 27.604 Acc@5 52.104
************train_loss 3.765541959609225 val_acc 27.604324340820312*************
Epoch: [3][    0/13346]	Time  1.356 ( 1.356)	Data  1.241 ( 1.241)	Loss 3.1416e+00 (3.1416e+00)	Acc@1  35.42 ( 35.42)	Acc@5  57.29 ( 57.29)
 * Acc@1 30.139 Acc@5 55.784
************train_loss 3.592500311117488 val_acc 30.139230728149414*************
Epoch

In [ ]:
epoch

In [ ]:
checkpoint = torch.load('../trained_model/gated/checkpoint.pt')

In [ ]:
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
